<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/Extract_disease_age_localColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**指定した病名の含まれている行を抽出して新規リストとして出力するスクリプト**

In [1]:
import csv
import numpy as np
import os
import os.path
import sys
import glob
import pandas as pd
import codecs

In [2]:
#xからyまでの数列を作る
def createList(x,y):
    lst = []
    for i in range(x,y+1):
        lst.append(i)
    return lst

data_path = r'C:\Users\ykita\FacePhoto\DiseaseInfo_all_connected.csv'
min_age = 0
max_age = 150
column = createList(5,24) #5行目から24行目までの病名を探索


In [ ]:
#CSV fileを開く
#文字化けする時は、utf8, shift-jis, cp932のいずれかに変更
with codecs.open(data_path, "r", "utf8", "ignore") as file:
    df_orig = pd.read_csv(file, delimiter=",",  header=None)
df_orig

In [7]:
#modules
def extract(byoumei, df_orig):
    #もとのcsvと列名を揃えたデータフレームを作成
    cols =  df_orig.columns
    df_dst = pd.DataFrame(index=[], columns=cols)
    for i in column:
        df_extract = (df_orig[df_orig[i].str.contains(byoumei, na=False)])
        df_dst = pd.concat([df_extract, df_dst])
    df_dst = df_dst[~df_dst.duplicated()]
    df_dst.sort_values(by=2,ascending=True, inplace=True)
    return df_dst

def sort_by_age(min_age, max_age, df):
    df_dst = df[(df[3]>=min_age) & (df[3]<=max_age)] #3列目が年齢に該当
    return df_dst

#**病名でソート**

In [8]:
byoumei = '外斜視'

df_dst = extract(byoumei, df_orig)
#print(df_dst)
print(len(df_dst))

2419


In [ ]:
dst_path = r'C:\Users\ykita\FacePhoto\DiseaseInfo_extracted_'+byoumei+'.csv'
df_dst.to_csv(dst_path, encoding='utf_8_sig', index = False, header=False)

#**年齢でソート**

In [9]:
min_age = 0
max_age = 150

df_dst = sort_by_age(min_age, max_age,df_orig)
print(len(df_dst))

9825


In [ ]:
dst_path = r'C:\Users\ykita\FacePhoto\DiseaseInfo_ageSorted_'+str(min_age)+'to'+str(max_age)+'.csv'
df_dst.to_csv(dst_path, encoding='utf_8_sig', index = False, header=False)

#**病名かつ年齢でソート**

In [13]:
byoumei = '外斜視'
min_age = 21
max_age = 100

df_dst = extract(byoumei, df_orig)
df_dst = sort_by_age(min_age, max_age,df_dst)
print(len(df_dst))


1304


In [14]:
dst_path = r'C:\Users\ykita\FacePhoto\DiseaseInfo_extracted_'+byoumei+'_'+str(min_age)+'to'+str(max_age)+'.csv'
df_dst.to_csv(dst_path, encoding='utf_8_sig', index = False, header=False)

#**病名が含まれていないものを抜き出し（年齢縛りあり）**

In [ ]:
def reverse_extract(byoumei, df_orig):
    #もとのcsvと列名を揃えたデータフレームを作成
    cols =  df_orig.columns
    df_byoumei = pd.DataFrame(index=[], columns=cols)
    #まず病名が含まれるdataframeを抜き出す
    for i in column:
        df_extract = (df_orig[df_orig[i].str.contains(byoumei, na=False)]) #文字列が含まれていないものを抜き出し
        df_byoumei = pd.concat([df_extract, df_byoumei])
    df_byoumei = df_byoumei[~df_byoumei.duplicated()]
    df_dst = df_orig[~df_orig[2].isin(df_byoumei[2])]
    #df_dst.sort_values(by=2,ascending=True, inplace=True)
    return df_dst

df_dst = reverse_extract("内斜視", df_orig)
df_dst = reverse_extract("外斜視", df_dst)
df_dst = sort_by_age(11,20,df_dst)

print(len(df_dst))

478


In [ ]:
dst_path = r'C:\Users\ykita\FacePhoto\DiseaseInfo_extracted_'+'内斜視外斜視以外_11to20'+'.csv'
df_dst.to_csv(dst_path, encoding='utf_8_sig', index = False, header=False)